In [ ]:
# Work on reimplementing Andrej Karpathy's nano-gpt from this video
# https://youtu.be/kCc8FmEb1nY?si=cdnNJrPwDEPQPn0C

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline